In [ ]:
# General Imports
import numpy as np
import os
from PIL import Image

# Plotly Imports
import plotly.graph_objects as go
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode,plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

#NHL_plot_utils import from src directory
import importlib.util

parent_dir = os.path.abspath(os.path.join('..'))
plot_utils_path = os.path.join(parent_dir, 'src', 'NHL_plot_utils.py')

spec = importlib.util.spec_from_file_location('NHL_plot_utils', plot_utils_path)
NHL_plot_utils = importlib.util.module_from_spec(spec)
spec.loader.exec_module(NHL_plot_utils)

all_team_avg = NHL_plot_utils.all_team_avg
transform_coord = NHL_plot_utils.transform_coord

print(f'Module imported from: {plot_utils_path}')

#Load cufflinks
%pip install cufflinks


In [2]:
FIGURE_DIR = os.path.join('..','figures')
RINK_IMG  = os.path.join(FIGURE_DIR,'nhl_rink.png')

In [3]:
import numpy as np
import plotly.graph_objects as go
from PIL import Image

# Update team names using the provided dictionary
nhl_teams = {
    1: "New Jersey Devils",
    2: "New York Islanders",
    3: "New York Rangers",
    4: "Philadelphia Flyers",
    5: "Pittsburgh Penguins",
    6: "Boston Bruins",
    7: "Buffalo Sabres",
    8: "Montreal Canadiens",
    9: "Ottawa Senators",
    10: "Toronto Maple Leafs",
    12: "Carolina Hurricanes",
    13: "Florida Panthers",
    14: "Tampa Bay Lightning",
    15: "Washington Capitals",
    16: "Chicago Blackhawks",
    17: "Detroit Red Wings",
    18: "Nashville Predators",
    19: "St. Louis Blues",
    20: "Calgary Flames",
    21: "Colorado Avalanche",
    22: "Edmonton Oilers",
    23: "Vancouver Canucks",
    24: "Anaheim Ducks",
    25: "Dallas Stars",
    26: "Los Angeles Kings",
    28: "San Jose Sharks",
    29: "Columbus Blue Jackets",
    30: "Minnesota Wild",
    52: "Winnipeg Jets",
    53: "Arizona Coyotes",
    54: "Vegas Golden Knights",
    55: "Seattle Kraken"
}

def contour(df_shoot, html_out=True):
    # Find all unique teams in the dataset
    teams_col = []
    
    for year in df_shoot.keys():
        for team in df_shoot[year].keys():
            if team not in teams_col:
                teams_col.append(team)
    
    # Create a default NaN matrix for teams missing in certain seasons
    A = np.ones((100, 85))
    A[:] = np.nan
    df = {}
    
    for year in df_shoot.keys():
        df[year] = {}
        for i in teams_col:
            if i not in df_shoot[year].keys():
                df_shoot[year][i] = A
            df[year][i] = df_shoot[year][i]

    # Prepare the x and y grid for the plot
    xx, yy = np.mgrid[0:100:100j, -42.5:42.5:85j]
    tiny_year_init = df_shoot[list(df.keys())[0]]
    tiny_team_init = tiny_year_init[list(tiny_year_init.keys())[0]]
    first_year = list(df_shoot.keys())[0]
    first_team = list(df_shoot[first_year].keys())[0]

    fig = go.Figure()

    # Add rink image
    fig.add_layout_image(
        dict(
            source=Image.open(RINK_IMG),
            xref="x",
            yref="y",
            x=-100,
            y=42.5,
            sizex=200,
            sizey=85,
            sizing="stretch",
            opacity=0.5,
            layer="above"
        )
    )

    # Add the initial contour plot for the first team and year
    fig.add_trace(
        go.Contour(
            x=xx[:, 1],
            y=yy[1, :],
            z=np.rot90(np.fliplr(df_shoot[first_year][first_team])),
            colorscale='Viridis',
            reversescale=True,
            connectgaps=False,
            name=nhl_teams.get(first_team, f"Team {first_team}"),
            colorbar=dict(
                title="DensitÃ© de tirs",
                titleside="right",
                tickmode="auto",
                tickvals=np.arange(-0.006, +0.008, 0.001),
                ticktext=[f'{val:.1f}' for val in np.arange(-0.006, +0.008, 0.001)],
                tickfont=dict(size=12, color="black"),
                ticks="outside",
                len=0.75
            ),
            zmin=-0.006,
            zmax=0.006
        )
    )

    # Create dropdown menus for years and teams
    updatemenu = []
    buttons = []
    buttons2 = []


    current_year = list(df.keys())[0]  # Initialize to first year
    current_team_id = teams_col[0]      # Initialize to first team

    # Year buttons
    # Inside your Year buttons loop
    
    for year in df.keys():
        buttons.append(dict(
           method='update',
           label=str(year),  # Convert year to string
           visible=True,
           args=[{'z': [np.rot90(np.fliplr(df[year][current_team_id]))]},  # Update z to show selected team's data for the year
              {'name': [nhl_teams.get(current_team_id, f"Team {current_team_id}")]},  # Update name to selected team
              ]
    ))
    
    # Team buttons
    # Inside your Team buttons loop
    for i, team_id in enumerate(teams_col):
        team_label = nhl_teams.get(team_id, f"Team {team_id}")
        buttons2.append(dict(
           method='restyle',
           label=team_label,
           args=[{'z': [np.rot90(np.fliplr(df[current_year][team_id]))]},  # Reference current year and selected team
              {'name': [team_label]}]
    ))
    # Configure update menus
    updatemenu = []
    updatemenu.append(dict(
        buttons=buttons,
        direction="down",
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.09,
        xanchor="left",
        y=1.2,
        yanchor="top"
    ))
    updatemenu.append(dict(
        buttons=buttons2,
        direction="down",
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.4,
        xanchor="left",
        y=1.2,
        yanchor="top"
    ))

    # Add the dropdown menus to the figure
    fig.update_layout(showlegend=False, updatemenus=updatemenu)

    # Add title and annotations
    fig.update_layout(
        title=dict(
            text="<i> Shot Frequency of Team Compared to League Average for the Season </i>",
            font={'size': 18},
            y=0.075,
            x=0.5,
            xanchor='center',
            yanchor='top'
        )
    )
    fig.update_layout(
        annotations=[
            go.layout.Annotation(text="year", x=0.01, xref="paper", y=1.15, yref="paper",
                                 align="left", showarrow=False),
            go.layout.Annotation(text="team", x=0.36, xref="paper", y=1.15,
                                 yref="paper", showarrow=False),
        ]
    )

    # Show the figure
    fig.show()

    # Output HTML if required
    if html_out:
        fig.update_layout(
            title=dict(
                text="<i> Shot Frequency of Team Compared to League Average for the Season </i>",
                font={'size': 18},
                y=0.075,
                x=0.5,
                xanchor='center',
                yanchor='top'
            )
        )
        fig.write_html(os.path.join(FIGURE_DIR, 'NHL_plot.html'), default_width='90%', default_height="100%", include_plotlyjs="cdn")

In [ ]:
df = transform_coord()
team_year_shoot = all_team_avg(df, start_year = 2016, end_year = 2020)
contour(team_year_shoot)